In [0]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

In [0]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [0]:
!ls -l

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving noisy_deep_features.npz to noisy_deep_features (1).npz
User uploaded file "noisy_deep_features.npz" with length 2463422 bytes


In [0]:
npzfile = np.load('noisy_deep_features.npz')
X_train = npzfile['arr_0']
X_val = npzfile['arr_1']
y_train = npzfile['arr_2']
y_val = npzfile['arr_3']

In [0]:
npzfile = np.load('noisy_deepfeat_full.npz')
noisy_data = npzfile['arr_0']
noisy_labels = npzfile['arr_1']


npzfile = np.load('studio_deepfeat_full.npz')
studio_recorded_features = npzfile['arr_0']
studio_recorded_labels = npzfile['arr_1']

In [0]:
studio_recorded_features, studio_recorded_labels

In [0]:
#studio_noisy_feat = []
studio_noisy_feat = np.append(studio_recorded_features,noisy_data, axis=0)
studio_noisy_labels = np.append(studio_recorded_labels,noisy_labels, axis=0)

In [62]:
studio_noisy_feat.shape

(20262, 50)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(studio_noisy_feat, studio_noisy_labels, random_state=42, test_size=0.20)

In [0]:
X = X_train
y = y_train

In [67]:
X.shape

(16209, 50)

In [0]:
param_grid = { 'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(128,64), (64,32), (128,64, 32),(64,32,16), (512, 256, 64)],

'activation': ["relu", "Tanh"],
             'solver': ["lbfgs","adam"]}
#'alpha': [10.0 ** -np.arange(1, 7)],

In [0]:
### hyper parameters
# activation
a = ["identity","logistic","tanh","relu"]
#a = ["tanh","relu"]
#solvers
s = ["lbfgs","sgd","adam"]
#s = ["adam"]
#learning rate
lr = [0.0001,0.001,0.01,0.1]
#hidden layers
#h = [(5,2),(3,2),(6,3),(7,2)]
h = [(128,64), (64,32), (128,64, 32),(64,32,16), (512, 256, 64)]

In [0]:
kf = KFold(n_splits=4)

In [0]:
def mlp(a,s,h,lr):
    clf = MLPClassifier(activation= a ,solver= s ,hidden_layer_sizes = h,max_iter = 5000 ,learning_rate = 'invscaling',learning_rate_init=lr)
    return clf  
#function to calculate the accuracy
def accuracy(actual,predicted):
    return np.count_nonzero(actual == predicted)*1.0/len(actual)

In [0]:
test_accuracy = []
train_accuracy = []

for i in range(10):
    k1 = np.random.randint(0,len(a))
    k2 = np.random.randint(0,len(s))
    k3 = np.random.randint(0,len(lr))
    k4 = np.random.randint(0,len(h))
    print("\nHyper-parameters = \n activation = ", a[k1],    "\n solver = ", s[k2], "\n learning_rate_init = ", lr[k3],         "\n hidden_layer_sizes = ", h[k4])
     #calling the mlp function with random hyper paramters
    clf = mlp(a[k1],s[k2],h[k4],lr[k3])
    tempTrain = 0
    tempTest = 0
    for train_index, test_index in kf.split(X):
        ## Splitting the data into train and test
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test  = y[train_index], y[test_index]
        ##fit the data into the model
        clf.fit(X_train,Y_train)
        ##predicting the values on the fitted model using train data
        predTrain = clf.predict((X_train))
        #adding the accuracy
        tempTrain = tempTrain + accuracy(Y_train,predTrain)
        ##predict the values on the fitted model using test data
        predTest = clf.predict(X_test)
        #adding the accuracy
        tempTest = tempTest + accuracy(Y_test, predTest)
        print("(train,test) accuracy = ", accuracy(Y_train,predTrain), accuracy(Y_test, predTest))
    ##Calculating the train accuracy
    train_accuracy.append(tempTrain*1.0/4)
    ##Calculating the test accuracy
    test_accuracy.append(tempTest*1.0/4)
    print("(train,test) accuracy = ",tempTrain*1.0/4, tempTest*1.0/4)

In [0]:
clf = MLPClassifier(max_iter = 5000)

In [34]:
X_train.shape

(9471, 50)

In [0]:
grid = GridSearchCV(clf, param_grid, verbose=2, n_jobs=4)

grid.fit(X_train, y_train)
print(grid.best_params_)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  5.4min
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 50.9min


In [1]:
print(grid.best_params_)

NameError: ignored